In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Demo_Transform") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/25 06:47:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
CLEAN_DATA_PATH = "/opt/bitnami/spark/resources/dataset/cleaned_data"

df = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(CLEAN_DATA_PATH)

df.limit(5).toPandas()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,overview,popularity,genres,production_companies,spoken_languages,keywords,production_countries
0,5,Four Rooms,5.784,2436,Released,1995-12-09,4257354,98,False,4000000,en,It's Ted the Bellhop's first night on the job....,15.295,Comedy,"Miramax, A Band Apart",English,"hotel, new year's eve, witch, bet, sperm, hote...",united states of america
1,6,Judgment Night,6.533,302,Released,1993-10-15,12136938,109,False,21000000,en,"While racing to a boxing match, Frank, Mike, J...",13.564,"Action, Crime, Thriller","Largo Entertainment, JVC",English,"drug dealer, chicago, illinois, escape, one ni...",united states of america
2,12,Finding Nemo,7.824,18061,Released,2003-05-30,940335536,100,False,94000000,en,"Nemo, an adventurous young clownfish, is unexp...",55.456,"Animation, Family",Pixar,English,"sydney, australia, parent child relationship, ...",united states of america
3,13,Forrest Gump,8.477,25409,Released,1994-06-23,677387716,142,False,55000000,en,A man with a low IQ has accomplished great thi...,92.693,"Comedy, Drama, Romance","Paramount, The Steve Tisch Company, Wendy Fine...",English,"vietnam war, vietnam veteran, mentally disable...",united states of america
4,15,Citizen Kane,8.015,5034,Released,1941-04-17,23218000,119,False,839727,en,"Newspaper magnate, Charles Foster Kane is take...",28.218,"Mystery, Drama","Mercury Productions, RKO Radio Pictures",English,"media tycoon, florida, art collector, newspape...",united states of america


In [4]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: timestamp (nullable = true)
 |-- revenue: integer (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- budget: integer (nullable = true)
 |-- original_language: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- production_countries: string (nullable = true)



In [5]:
print((df.count(), len(df.columns)))

(8441, 18)


In [7]:
# Create Dim Movie
dim_movie_df = df.select("id", "status", "release_date", "adult", "original_language", "overview") \
                 .withColumnRenamed("id", "movie_id")

dim_movie_df.show(5)

+--------+--------+-------------------+-----+-----------------+--------------------+
|movie_id|  status|       release_date|adult|original_language|            overview|
+--------+--------+-------------------+-----+-----------------+--------------------+
|       5|Released|1995-12-09 00:00:00|false|               en|It's Ted the Bell...|
|       6|Released|1993-10-15 00:00:00|false|               en|While racing to a...|
|      12|Released|2003-05-30 00:00:00|false|               en|Nemo, an adventur...|
|      13|Released|1994-06-23 00:00:00|false|               en|A man with a low ...|
|      15|Released|1941-04-17 00:00:00|false|               en|Newspaper magnate...|
+--------+--------+-------------------+-----+-----------------+--------------------+
only showing top 5 rows



In [8]:
dim_movie_df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: timestamp (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- original_language: string (nullable = true)
 |-- overview: string (nullable = true)



In [9]:
fact_movie_df = df.select("id", "vote_average", "popularity", "vote_count", "budget", "revenue", "runtime") \
                  .withColumnRenamed("id", "movie_id")

fact_movie_df.show(5)

+--------+------------+----------+----------+--------+---------+-------+
|movie_id|vote_average|popularity|vote_count|  budget|  revenue|runtime|
+--------+------------+----------+----------+--------+---------+-------+
|       5|       5.784|    15.295|      2436| 4000000|  4257354|     98|
|       6|       6.533|    13.564|       302|21000000| 12136938|    109|
|      12|       7.824|    55.456|     18061|94000000|940335536|    100|
|      13|       8.477|    92.693|     25409|55000000|677387716|    142|
|      15|       8.015|    28.218|      5034|  839727| 23218000|    119|
+--------+------------+----------+----------+--------+---------+-------+
only showing top 5 rows



In [10]:
fact_movie_df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- popularity: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- budget: integer (nullable = true)
 |-- revenue: integer (nullable = true)
 |-- runtime: integer (nullable = true)



In [32]:
from pyspark.sql.functions import explode, split, trim, col, monotonically_increasing_id

# dim_genres = df.select("genres").withColumnRenamed("genres", "genre_name")
# dim_genres = dim_genres.withColumn("genre_name", split("genre_name", ", "))
# dim_genres = dim_genres.withColumn("genre_name", explode("genre_name"))
# dim_genres = dim_genres.select(trim("genre_name")).distinct()

# Create distinct genre names
dim_genres_df = (
    df.select(explode(split(col("genres"), ",\s*")).alias("genre_name"))
      .select(trim(col("genre_name")).alias("genre_name"))
      .distinct()
)

# Add genre_id using monotonically_increasing_id
dim_genres_df = dim_genres_df.withColumn("genre_id", monotonically_increasing_id())

# Optional: reorder columns
dim_genres_df = dim_genres_df.select("genre_id", "genre_name")

dim_genres_df.show(5)

+--------+----------+
|genre_id|genre_name|
+--------+----------+
|       0|     Crime|
|       1|   Romance|
|       2|  TV Movie|
|       3|  Thriller|
|       4| Adventure|
+--------+----------+
only showing top 5 rows



In [34]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import explode, split, trim, col, monotonically_increasing_id

def create_dim_table_from_column(df: DataFrame, column_name: str, delimiter: str = ",", id_column: str = "id", value_column: str = "value") -> DataFrame:
    """
    Create a dimension table with distinct values from a delimited string column,
    and assign a unique ID using monotonically_increasing_id.

    Parameters:
    - df: Input Spark DataFrame
    - column_name: Name of the column to extract distinct values from
    - delimiter: Delimiter to split the values (default is comma)
    - id_column: Name for the generated ID column (default is 'id')
    - value_column: Name for the value column (default is 'value')

    Returns:
    - A DataFrame with columns [id_column, value_column]
    """
    return (
        df.select(explode(split(col(column_name), delimiter + r"\s*")).alias(value_column))
          .select(trim(col(value_column)).alias(value_column))
          .distinct()
          .withColumn(id_column, monotonically_increasing_id())
          .select(id_column, value_column)
    )

In [35]:
dim_genres_df = create_dim_table_from_column(df, "genres", id_column="genre_id", value_column="genre_name")
dim_genres_df.show(5)

+--------+----------+
|genre_id|genre_name|
+--------+----------+
|       0|     Crime|
|       1|   Romance|
|       2|  TV Movie|
|       3|  Thriller|
|       4| Adventure|
+--------+----------+
only showing top 5 rows



In [36]:
dim_keyword_df = create_dim_table_from_column(
    df, "keywords", id_column="keyword_id", value_column="keyword_name"
)

dim_keyword_df.show(5)

+----------+------------+
|keyword_id|keyword_name|
+----------+------------+
|         0|       1970s|
|         1| pirate ship|
|         2|       anime|
|         3|inflammatory|
|         4|  hoverboard|
+----------+------------+
only showing top 5 rows



In [37]:
dim_keyword_df.count()

15225

In [40]:
dim_production_company_df = create_dim_table_from_column(
    df, "production_companies", id_column="company_id", value_column="company_name"
)

dim_production_company_df.show(5)

+----------+--------------------+
|company_id|        company_name|
+----------+--------------------+
|         0|                 PEA|
|         1|            ZDF/Arte|
|         2|    Relativity Media|
|         3|    Koppelman-Levien|
|         4|Win's Entertainme...|
+----------+--------------------+
only showing top 5 rows



In [41]:
dim_production_company_df.count()

8950

In [42]:
dim_production_country_df = create_dim_table_from_column(
    df, "production_countries", id_column="country_id", value_column="country_name"
)

dim_production_country_df.show(5)

+----------+------------+
|country_id|country_name|
+----------+------------+
|         0|     finland|
|         1|   australia|
|         2|      greece|
|         3|    portugal|
|         4|      israel|
+----------+------------+
only showing top 5 rows



In [43]:
dim_production_country_df.count()

108

In [44]:
dim_spoken_language_df = create_dim_table_from_column(
    df, "spoken_languages", id_column="language_id", value_column="language_name"
)

dim_spoken_language_df.show(5)

+-----------+-------------+
|language_id|language_name|
+-----------+-------------+
|          0|      Lingala|
|          1|         Urdu|
|          2|        Khmer|
|          3|      Maltese|
|          4|          Ido|
+-----------+-------------+
only showing top 5 rows



In [45]:
dim_spoken_language_df.count()

114

# Create Bridge Table

In [64]:
from pyspark.sql.functions import explode, split, trim, col, monotonically_increasing_id

# dim_genres = df.select("genres").withColumnRenamed("genres", "genre_name")
# dim_genres = dim_genres.withColumn("genre_name", split("genre_name", ", "))
# dim_genres = dim_genres.withColumn("genre_name", explode("genre_name"))
# dim_genres = dim_genres.select(trim("genre_name")).distinct()

# Load 

# Create Function Bridge Table
bridge_movie_genre_df = df.select("id", "genres").withColumnRenamed("id", "movie_id")
bridge_movie_genre_df = bridge_movie_genre_df.withColumn("genres", split("genres", ", "))
bridge_movie_genre_df = bridge_movie_genre_df.withColumn("genres", explode("genres"))
bridge_movie_genre_df = bridge_movie_genre_df.select("movie_id", trim("genres").alias("genres"))
bridge_movie_genre_df = bridge_movie_genre_df.join(
                            dim_genres_df, 
                            bridge_movie_genre_df.genres == dim_genres_df.genre_name, 
                            "inner"
                        )
bridge_movie_genre_df = bridge_movie_genre_df.select("movie_id", "genre_id")
bridge_movie_genre_df = bridge_movie_genre_df.withColumn("bridge_id", monotonically_increasing_id())

# Reorder
bridge_movie_genre_df = bridge_movie_genre_df.select("bridge_id", "movie_id", "genre_id")

bridge_movie_genre_df.show(5)

+---------+--------+--------+
|bridge_id|movie_id|genre_id|
+---------+--------+--------+
|        0| 1139554|       0|
|        1| 1061412|       0|
|        2| 1014066|       0|
|        3|  958006|       0|
|        4|  919573|       0|
+---------+--------+--------+
only showing top 5 rows



In [65]:
bridge_movie_genre_df.count()

21580

In [60]:
bridge_movie_genre_df = bridge_movie_genre_df.join(
                            dim_genres_df, 
                            bridge_movie_genre_df.genres == dim_genres_df.genre_name, 
                            "inner"
                        )

bridge_movie_genre_df.show(5)

+--------+------+--------+----------+
|movie_id|genres|genre_id|genre_name|
+--------+------+--------+----------+
| 1139554| Crime|       0|     Crime|
| 1061412| Crime|       0|     Crime|
| 1014066| Crime|       0|     Crime|
|  958006| Crime|       0|     Crime|
|  919573| Crime|       0|     Crime|
+--------+------+--------+----------+
only showing top 5 rows



In [61]:
bridge_movie_genre_df.count()

21580

In [67]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import split, explode, trim, col, monotonically_increasing_id

def create_bridge_table(
    fact_df: DataFrame,
    dim_df: DataFrame,
    fact_id_col: str,
    fact_list_col: str,
    dim_value_col: str,
    dim_id_col: str,
    delimiter: str = ", ",
    bridge_id_col: str = "bridge_id"
) -> DataFrame:
    """
    Create a bridge (many-to-many) table between a fact table and a dimension table.

    Parameters:
    - fact_df: Source DataFrame containing list column (e.g. genres)
    - dim_df: Dimension DataFrame with distinct values and IDs
    - fact_id_col: Name of ID column in the fact table (e.g. movie_id)
    - fact_list_col: Name of delimited string column in the fact table (e.g. genres)
    - dim_value_col: Name of value column in the dimension table (e.g. genre_name)
    - dim_id_col: Name of ID column in the dimension table (e.g. genre_id)
    - delimiter: Delimiter to split the values (default: ', ')
    - bridge_id_col: Name of ID column to generate in the bridge table

    Returns:
    - DataFrame with bridge table [bridge_id, fact_id_col, dim_id_col]
    """
    exploded_df = (
        fact_df.select(col(fact_id_col), col(fact_list_col))
               .withColumn(fact_list_col, split(col(fact_list_col), delimiter))
               .withColumn(fact_list_col, explode(col(fact_list_col)))
               .select(col(fact_id_col), trim(col(fact_list_col)).alias(fact_list_col))
    )

    joined_df = (
        exploded_df.join(
            dim_df,
            exploded_df[fact_list_col] == dim_df[dim_value_col],
            "inner"
        )
    )

    bridge_df = (
        joined_df.select(col(fact_id_col), col(dim_id_col))
                 .withColumn(bridge_id_col, monotonically_increasing_id())
                 .select(bridge_id_col, fact_id_col, dim_id_col)
    )

    return bridge_df

In [77]:
bridge_movie_genre_df = create_bridge_table(
    fact_df=df,
    dim_df=dim_genres_df,
    fact_id_col="id",
    fact_list_col="genres",
    dim_value_col="genre_name",
    dim_id_col="genre_id",
    bridge_id_col="bridge_id"
)

bridge_movie_genre_df = bridge_movie_genre_df.withColumnRenamed("id", "movie_id")

bridge_movie_genre_df.show(5)

+---------+--------+--------+
|bridge_id|movie_id|genre_id|
+---------+--------+--------+
|        0| 1139554|       0|
|        1| 1061412|       0|
|        2| 1014066|       0|
|        3|  958006|       0|
|        4|  919573|       0|
+---------+--------+--------+
only showing top 5 rows



In [69]:
bridge_movie_genre_df.count()

21580

In [78]:
bridge_movie_keyword_df = create_bridge_table(
    fact_df=df,
    dim_df=dim_keyword_df,
    fact_id_col="id",
    fact_list_col="keywords",
    dim_value_col="keyword_name",
    dim_id_col="keyword_id",
    bridge_id_col="bridge_id"
)

bridge_movie_keyword_df = bridge_movie_keyword_df.withColumnRenamed("id", "movie_id")

bridge_movie_keyword_df.show(5)

+---------+--------+----------+
|bridge_id|movie_id|keyword_id|
+---------+--------+----------+
|        0|  878361|         0|
|        1|  760104|         0|
|        2|  756999|         0|
|        3|  718032|         0|
|        4|  555285|         0|
+---------+--------+----------+
only showing top 5 rows



In [73]:
bridge_movie_keyword_df.count()

81539

In [79]:
bridge_movie_company_df = create_bridge_table(
    fact_df=df,
    dim_df=dim_production_company_df,
    fact_id_col="id",
    fact_list_col="production_companies",
    dim_value_col="company_name",
    dim_id_col="company_id",
    bridge_id_col="bridge_id"
)

bridge_movie_company_df = bridge_movie_company_df.withColumnRenamed("id", "movie_id")

bridge_movie_company_df.show(5)

+---------+--------+----------+
|bridge_id|movie_id|company_id|
+---------+--------+----------+
|        0|    1643|         0|
|        1|     429|         0|
|        2|  888321|         1|
|        3|  336804|         1|
|        4|  126250|         1|
+---------+--------+----------+
only showing top 5 rows



In [76]:
bridge_movie_company_df.count()

27226

In [80]:
bridge_movie_country_df = create_bridge_table(
    fact_df=df,
    dim_df=dim_production_country_df,
    fact_id_col="id",
    fact_list_col="production_countries",
    dim_value_col="country_name",
    dim_id_col="country_id",
    bridge_id_col="bridge_id"
)

bridge_movie_country_df = bridge_movie_country_df.withColumnRenamed("id", "movie_id")
bridge_movie_country_df.show(5)

+---------+--------+----------+
|bridge_id|movie_id|country_id|
+---------+--------+----------+
|        0|  840326|         0|
|        1|  798286|         0|
|        2|  302349|         0|
|        3|  230179|         0|
|        4|  224746|         0|
+---------+--------+----------+
only showing top 5 rows



In [81]:
bridge_movie_country_df.count()

12080

In [82]:
bridge_movie_language_df = create_bridge_table(
    fact_df=df,
    dim_df=dim_spoken_language_df,
    fact_id_col="id",
    fact_list_col="spoken_languages",
    dim_value_col="language_name",
    dim_id_col="language_id",
    bridge_id_col="bridge_id"
)

bridge_movie_language_df = bridge_movie_language_df.withColumnRenamed("id", "movie_id")
bridge_movie_language_df.show(5)

+---------+--------+-----------+
|bridge_id|movie_id|language_id|
+---------+--------+-----------+
|        0|    1247|          0|
|        1|  544402|          1|
|        2|  543005|          1|
|        3|  495429|          1|
|        4|  473309|          1|
+---------+--------+-----------+
only showing top 5 rows



In [83]:
bridge_movie_language_df.count()

12477

In [85]:
spark.stop()